In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [3]:
imagesize = [224,224]

data =r"/content/drive/MyDrive/Yoga Pose/Dataset"

In [4]:
res = ResNet50(include_top=False, input_shape=imagesize+[3],weights="imagenet")

94781440/94765736 [==============================] - 0s 0us/step


In [5]:
for layer in res.layers:
  layer.trainable = False

In [6]:
x = Flatten()(res.output)

In [7]:
prediction = Dense(5, activation='softmax')(x)

In [8]:
model = Model(inputs = res.input, outputs = prediction)

In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [10]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
    )

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split = 0.3)

In [12]:
training_set = train_datagen.flow_from_directory(data,
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                 subset='training')

testing_set = train_datagen.flow_from_directory(data,
                                                target_size = (224,224),
                                                batch_size = 32,
                                                class_mode = 'categorical',
                                                subset='validation')



Found 694 images belonging to 5 classes.
Found 294 images belonging to 5 classes.


In [13]:
training_set.class_indices

{'Downdog': 0, 'Goddess': 1, 'Plank': 2, 'Tree': 3, 'Warrior2': 4}

In [14]:
len(training_set)

22

In [15]:
r = model.fit_generator(
    training_set,
    validation_data = testing_set,
    epochs = 40,
    steps_per_epoch = len(training_set)//2,
    validation_steps = len(testing_set)//2
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/40
11/11 [==============================] - 95s 8s/step - loss: 9.3252 - accuracy: 0.2188 - val_loss: 7.0537 - val_accuracy: 0.2625
Epoch 2/40
11/11 [==============================] - 39s 4s/step - loss: 6.1705 - accuracy: 0.2670 - val_loss: 3.7051 - val_accuracy: 0.2625
Epoch 3/40
11/11 [==============================] - 23s 2s/step - loss: 2.5869 - accuracy: 0.3466 - val_loss: 1.5975 - val_accuracy: 0.4313
Epoch 4/40
11/11 [==============================] - 14s 1s/step - loss: 1.6775 - accuracy: 0.4489 - val_loss: 1.5573 - val_accuracy: 0.4688
Epoch 5/40
11/11 [==============================] - 11s 970ms/step - loss: 1.1301 - accuracy: 0.5426 - val_loss: 0.6098 - val_accuracy: 0.7188
Epoch 6/40
11/11 [==============================] - 9s 859ms/step - loss: 1.0211 - accuracy: 0.6053 - val_loss: 0.6143 - val_accuracy: 0.7500
Epoch 7/40
11/11 [==============================] - 8s 723ms/step - loss: 0.9649 - accuracy: 0.6278 - val_loss: 0.6677 - val_accuracy: 0.7312
Epoch 8/40
11

In [16]:
model.save('resnet50_yoga.h5')

In [17]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input


In [18]:
model = load_model("resnet50_yoga.h5")

In [19]:
img = load_img("/content/drive/MyDrive/Yoga Pose/Dataset/Goddess/00000019.jpg", target_size=(224,224))

In [20]:
x = image.img_to_array(img)

In [21]:
x.shape

(224, 224, 3)

In [22]:
x = np.expand_dims(x, axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [23]:
pred = model.predict(img_data)

In [24]:
pred

array([[4.8336861e-13, 8.2695490e-01, 3.7450496e-09, 2.3776056e-11,
        1.7304516e-01]], dtype=float32)

In [25]:
output = np.argmax(pred)

In [26]:
output

1

In [27]:
columns = ['Downdog', 'Goddess', 'Plank', 'Tree', 'Warrior2']

In [28]:
result = str(columns[output])

In [29]:
result

'Goddess'